In [10]:
import sys

import beakerx
import ipywidgets
from IPython.display import display, HTML
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

sys.path.append('../..')

from solution import solarpvutil

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

scenario_choices = ipywidgets.SelectMultiple(
    options=solarpvutil.scenarios,
    description='Scenarios',
    disabled=False, rows=8
)
scenario_choices

SelectMultiple(description='Scenarios', options=('PDS3-16p2050-Optimum (Updated)', 'PDS2-15p2050-Drawdown (Upd…

In [15]:
#%%prun -s cumulative -l 200
if len(scenario_choices.value):
    spv = [solarpvutil.SolarPVUtil(v) for v in scenario_choices.value]
else:
    spv = [solarpvutil.SolarPVUtil()]

display(HTML('<h2>' + str(spv[0].name) + '</h2>'))

# ------------------ Detailed Results -----------------

def blue_label(text):
    o = ipywidgets.Output()
    div = '<div style="background-color:LightSkyBlue;border:1px solid black;font-weight:bold;text-align:center;font-size:0.875em;">'
    with o:
        display(HTML(div + text + '</div>'))
    return o


key_results_label = blue_label('The Key Adoption Results')

# --- Unit Adoption ---

unit_adoption_text = []
for s in spv:
    pds_vs_ref = s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'] - s.ua.soln_ref_tot_iunits_reqd().loc[2050, 'World']
    unit_adoption_text.append([s.scenario, pds_vs_ref, s.ua.soln_net_annual_funits_adopted().loc[2050, 'World'] ])
unit_adoption_text.append(['', 'TW', 'TWh'])
unit_adoption_columns = ['Scenario', 'Implementation Unit Adoption Increase in 2050 (PDS vs REF)',
                         'Functional Unit Adoption Increase in 2050 (PDS vs REF)']
unit_adoption_heading = ipywidgets.Output()
with unit_adoption_heading:
    display(HTML(pd.DataFrame(unit_adoption_text, columns=unit_adoption_columns).to_html(index=False, float_format=lambda x: '%.3f' % x)))

# --- Adoption ---

adoption_text = []
for s in spv:
    adoption_text.append([s.scenario,
                          s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'],
                          s.ht.soln_pds_funits_adopted().loc[2050, 'World'],
                          '', '', ''])
adoption_text.append(['', 'TW', 'TWh', '%', '%', '%'])
adoption_columns = ['Scenario', 'Global Solution Implementation Units in 2050',
                    'Global Solution Functional Units in 2050', 'Global Solution Adoption Share - 2014',
                    'Global Solution Adoption Share - 2020', 'Global Solution Adoption Share - 2050']
adoption_heading = ipywidgets.Output()
with adoption_heading:
    display(HTML(pd.DataFrame(adoption_text, columns=adoption_columns).to_html(index=False, float_format=lambda x: '%.2f' % x)))

adoption_graphs = ipywidgets.Output()
with adoption_graphs:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    ax[0].set_title('World Adoption - Implementation Units (TW)')
    for s in spv:
        s.ua.soln_pds_tot_iunits_reqd()['World'].plot(ax=ax[0], legend=True, label=s.scenario, xlim=(2020, 2050))
    ax[1].set_title('World Growth of Solution - Functional Units (TWh)')
    for s in spv:
        s.tm.ref_tam_per_region()['World'].plot(ax=ax[1], legend=True, label='Total Market-REF:'+s.scenario, xlim=(2020, 2050))
        s.tm.pds_tam_per_region()['World'].plot(ax=ax[1], legend=True, label='Total Market-PDS:'+s.scenario, xlim=(2020, 2050))
        s.ht.soln_ref_funits_adopted()['World'].plot(ax=ax[1], legend=True, label='Solution-REF:'+s.scenario, xlim=(2020, 2050))
        s.ht.soln_pds_funits_adopted()['World'].plot(ax=ax[1], legend=True, label='Solution-PDS:'+s.scenario, xlim=(2020, 2050))
    plt.show(fig)

# --- Financial ---

financial_results_label = blue_label('The Key Financial Results</div>')

financial_text = []
for s in spv:
    marginal_first_cost = (s.fc.soln_pds_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.soln_ref_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.conv_ref_annual_world_first_cost().loc[:2050].sum()) / 10**9
    net_operating_savings = ((s.oc.conv_ref_cumulative_operating_cost().loc[2050] - \
                             s.oc.conv_ref_cumulative_operating_cost().loc[2020]) - \
                            (s.oc.soln_pds_cumulative_operating_cost().loc[2050] - \
                             s.oc.soln_pds_cumulative_operating_cost().loc[2020])) / 10**9
    lifetime_operating_savings = s.oc.lifetime_cost_forecast()['Marginal Operating Cost Savings'].sum() / 10**9
    financial_text.append([s.scenario, marginal_first_cost, net_operating_savings, lifetime_operating_savings])
financial_text.append(['', 'Billion USD', 'Billion USD', 'Billion USD'])
financial_columns = ['Scenario', 'Marginal First Cost 2020-2050',
                     'Net Operating Savings 2020-2050', 'Lifetime Operating Savings 2020-2050']

financial_heading = ipywidgets.Output()
with financial_heading:
    display(HTML(pd.DataFrame(financial_text, columns=financial_columns).to_html(index=False, float_format=lambda x: '%.2f' % x)))

cost_graphs = ipywidgets.Output()
with cost_graphs:
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 4))
    ax[0].set_title('World Operating Cost Difference')
    ax[0].set_xlim(left=2010, right=2060)
    ax[0].set_xlabel('Billions US$2014')
    matplotlib.ticker.EngFormatter.ENG_PREFIXES[9] = 'B'
    fmty = matplotlib.ticker.EngFormatter(sep='')
    ax[0].yaxis.set_major_formatter(fmty)
    for s in spv:
        linestyle = '--' if 'Plausible' in s.scenario else '-.' if 'Drawdown' in s.scenario else ':'
        s.oc.soln_pds_annual_operating_cost().loc[2020:2050].plot(ax=ax[0], legend=True, linestyle=linestyle, color='r', label='Solution Operating Costs/Savings')
        s.oc.conv_ref_annual_operating_cost().loc[2020:2050].plot(ax=ax[0], legend=True, linestyle=linestyle, color='g', label='Conventional Operating Costs/Savings')
    plt.show(fig)

# --- Climate ---

climate_results_label = blue_label('The Key Climate Results</div>')

climate_text = []
for s in spv:
    climate_text.append([s.scenario, (s.c2.co2eq_mmt_reduced().loc[:, 'World'].sum() / 1000)])
climate_text.append(['', 'Gt CO2 (2020-2050)'])
climate_columns = ['Scenario', 'Total Emissions Reduction']

climate_heading = ipywidgets.Output()
with climate_heading:
    display(HTML(pd.DataFrame(climate_text, columns=climate_columns).to_html(index=False, float_format=lambda x: '%.2f' % x)))

emissions_graphs = ipywidgets.Output()
with emissions_graphs:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(13, 4))
    ax[0].set_title('World Cumulative Emissions Reductions (Gigatons CO2-eq)')
    ax[0].set_xlim(left=2020, right=2050)
    for s in spv:
        s.c2.co2eq_mmt_reduced()['World'].cumsum().mul(0.001).plot(ax=ax[0], legend=True, label=s.scenario)
    ax[1].set_title('World Cumulative GHG Concentrations Reduction (PPM CO2-eq)')
    ax[1].set_xlim(left=2020, right=2050)
    for s in spv:
        s.c2.co2eq_ppm_calculator()['CO2-eq PPM'].plot(ax=ax[1], legend=True, label=s.scenario)
    plt.show(fig)

detailed_results = ipywidgets.Output()
with detailed_results:
    display(ipywidgets.VBox([key_results_label, unit_adoption_heading, adoption_heading, adoption_graphs,
                             financial_results_label, financial_heading, cost_graphs,
                             climate_results_label, climate_heading, emissions_graphs]))

# --------------- Variable Meta Analysis --------------

variable_meta_analysis = ipywidgets.Output()
with variable_meta_analysis:
    children = []
    for s in spv:
        for (name, v) in s.VMAs:
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(HTML(v.source_data.drop(['Link', 'Source Validation Code', 'License Code'], axis=1).fillna('').to_html()))
            accordion = ipywidgets.Accordion(children=[vma_widget])
            accordion.set_title(0, name)
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(ipywidgets.HBox(children=[ipywidgets.Label('Summary Placeholder'), accordion]))
            children.append(vma_widget)
    display(ipywidgets.VBox(children=children))

# ------------------ Operating Cost -----------------

children = []
for s in spv:
    df = pd.concat([s.oc.soln_pds_annual_operating_cost().loc[:2060],
                    s.oc.conv_ref_annual_operating_cost().loc[:2060],
                    s.oc.marginal_annual_operating_cost().loc[:2060]],
                   axis=1)
    oc_table = ipywidgets.Output()
    with oc_table:
        display(HTML(df.to_html()))
    children.append(oc_table)

operating_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(spv):
    operating_cost.set_title(i, s.scenario)

# ------------------ First Cost -----------------

children = []
for s in spv:
    df = pd.concat([s.fc.soln_pds_install_cost_per_iunit().loc[:2060],
                    s.fc.soln_ref_install_cost_per_iunit().loc[:2060],
                    s.fc.conv_ref_install_cost_per_iunit().loc[:2060]],
                   axis=1)
    fc_table = ipywidgets.Output()
    with fc_table:
        display(HTML(df.to_html()))
    children.append(fc_table)

first_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(spv):
    first_cost.set_title(i, s.scenario)

# ------------------ Create tabs -----------------

tabs = ipywidgets.Tab(children=[detailed_results, variable_meta_analysis, operating_cost, first_cost])
tabs.set_title(0, "Detailed Results")
tabs.set_title(1, "Variable Meta-Analysis")
tabs.set_title(2, "Operating Cost")
tabs.set_title(3, "First Cost")
display(tabs)